# Calculating Elder Impulse for long view

In [2]:
# IMPORTING PACKAGES

import pandas as pd
import requests
#import matplotlib.pyplot as plt
#from termcolor import colored as cl
#import math 
from datetime import datetime, timedelta
import sqlite3

#plt.rcParams['figure.figsize'] = (20,10)
#plt.style.use('fivethirtyeight')

/var/folders/sk/ckljlph97j1grnldzlf4khtc0000gn/T/ipykernel_18867/3088924878.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Getting historical data from binance API

In [3]:
#The variable days_back represents the number of days in the past from which I intend to retrieve the data.
#The variable symbol is a coin pair. For example if you want bitcoin historical data, the symbol to use is BTCUSDT. 

API_URL = 'https://api.binance.com/api/v3/klines' # Binance API endpoint for candlestick data

def get_historical_data(symbol, days_back, interval):
    # Calculate the start time in milliseconds
    start_time = int((datetime.now() - timedelta(days=days_back)).timestamp() * 1000)

    # Calculate the current time in milliseconds
    end_time = int(datetime.now().timestamp() * 1000)

    # Make an API request to retrieve historical data
    hist_json = requests.get(API_URL, params={
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'endTime': end_time,
        'limit': 1000  # Maximum number of data points per request
    }).json()

    #print(hist_json)

    # Making a df from response
    # Extract only the relevant elements for each row
    df = pd.DataFrame(hist_json, columns=[
        'Time', 'Open Price', 'High Price', 'Low Price',
        'Close Price', 'Volume', 'Kline Close Time', 'Quote Asset Volume',
        'Number of Trades', 'Taker Buy Base Asset Volume',
        'Taker Buy Quote Asset Volume', 'Unused Field'
    ])

    # Select only the relevant columns (Open Time, Open Price, High Price, Low Price, Close Price, Volume)
    df = df[['Time', 'Open Price', 'High Price', 'Low Price', 'Close Price', 'Volume']]
    df['dateTime'] = pd.to_datetime(df['Time'], unit='ms')
    
    return df


## Creating a table for calculating elder impulse

Elder Impulse
https://school.stockcharts.com/doku.php?id=chart_analysis:elder_impulse_system
MACD-histogram
https://www.investopedia.com/ask/answers/122414/what-moving-average-convergence-divergence-macd-formula-and-how-it-calculated.asp

In [4]:
df_long = get_historical_data('BTCUSDT', 30, '1w')


In [5]:
df_long.tail()


,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime
0,1705881600000,41580.32000000,42842.68000000,38555.00000000,42031.06000000,274603.16207000,2024-01-22
1,1706486400000,42031.05000000,43882.36000000,41804.88000000,42582.88000000,203036.61925000,2024-01-29
2,1707091200000,42582.88000000,48592.66000000,42258.10000000,48299.99000000,262240.48357000,2024-02-05
3,1707696000000,48300.00000000,52816.62000000,47710.01000000,51943.99000000,259111.38206000,2024-02-12


In [6]:
df_long['13EMA'] = df_long['Close Price'].ewm(span=13, adjust=False).mean()

In [7]:
df_long.tail()

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,13EMA
0,1705881600000,41580.32000000,42842.68000000,38555.00000000,42031.06000000,274603.16207000,2024-01-22,42031.060000
1,1706486400000,42031.05000000,43882.36000000,41804.88000000,42582.88000000,203036.61925000,2024-01-29,42109.891429
2,1707091200000,42582.88000000,48592.66000000,42258.10000000,48299.99000000,262240.48357000,2024-02-05,42994.191224
3,1707696000000,48300.00000000,52816.62000000,47710.01000000,51943.99000000,259111.38206000,2024-02-12,44272.733907


In [8]:
# Calculate 12-period EMA
df_long['12EMA'] = df_long['Close Price'].ewm(span=12, adjust=False).mean()

# Calculate 26-period EMA
df_long['26EMA'] = df_long['Close Price'].ewm(span=26, adjust=False).mean()

# Calculate MACD Line (12EMA - 26EMA)
df_long['MACD'] = df_long['12EMA'] - df_long['26EMA']

# Calculate Signal Line (9-period EMA of MACD)
df_long['Signal'] = df_long['MACD'].ewm(span=9, adjust=False).mean()

# Calculate MACD Histogram
df_long['Histogram'] = df_long['MACD'] - df_long['Signal']

In [9]:
df_long.tail()

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,13EMA,12EMA,26EMA,MACD,Signal,Histogram
0,1705881600000,41580.32000000,42842.68000000,38555.00000000,42031.06000000,274603.16207000,2024-01-22,42031.060000,42031.060000,42031.060000,0.000000,0.000000,0.000000
1,1706486400000,42031.05000000,43882.36000000,41804.88000000,42582.88000000,203036.61925000,2024-01-29,42109.891429,42115.955385,42071.935556,44.019829,8.803966,35.215863
2,1707091200000,42582.88000000,48592.66000000,42258.10000000,48299.99000000,262240.48357000,2024-02-05,42994.191224,43067.345325,42533.272922,534.072404,113.857653,420.214750
3,1707696000000,48300.00000000,52816.62000000,47710.01000000,51943.99000000,259111.38206000,2024-02-12,44272.733907,44432.982968,43230.363076,1202.619892,331.610101,871.009791


## Calculating elder impulse

In [10]:
last_row_13ema = df_long.iloc[-1]['13EMA']
second_to_last_row_13ema = df_long.iloc[-2]['13EMA']
# Print the comparison
print(f"Last row EMA_13: {last_row_13ema}")
print(f"Second-to-last row EMA_13: {second_to_last_row_13ema}")

Last row EMA_13: 44272.73390670554
Second-to-last row EMA_13: 42994.1912244898


In [11]:
ema_comparison = 0

if last_row_13ema > second_to_last_row_13ema:
    ema_comparison = 1
elif last_row_13ema < second_to_last_row_13ema:
    ema_comparison = -1

ema_comparison

1

In [12]:
elder_impulse = 0

if ema_comparison == 1 and df_long['Histogram'].iloc[-1] > 0:
    elder_impulse = 1
elif ema_comparison == -1 and df_long['Histogram'].iloc[-1] < 0:
    elder_impulse = -1
else:
    elder_impulse = 0

elder_impulse

1

if elder_impulse is 1, it means you can go long; 
if it is -1 , you can go short;
if it is 0 you can do both. 
It is best to go long if impulse is 1 or 0 after -1. (This should be considered good metricks for stock screening)

# Creating Medium view and ATR channels

In [13]:
df_medium = get_historical_data('BTCUSDT', 30, '1d')

In [14]:
df_medium.head()


,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime
0,1705536000000,42776.09000000,42930.00000000,40683.28000000,41327.50000000,43907.51641000,2024-01-18
1,1705622400000,41327.51000000,42196.86000000,40280.00000000,41659.03000000,48342.74559000,2024-01-19
2,1705708800000,41659.03000000,41872.56000000,41456.30000000,41696.04000000,15923.99493000,2024-01-20
3,1705795200000,41696.05000000,41881.39000000,41500.98000000,41580.33000000,11730.16301000,2024-01-21
4,1705881600000,41580.32000000,41689.65000000,39431.58000000,39568.02000000,55426.19911000,2024-01-22


In [15]:
import talib

In [16]:
df_medium['ATR21'] = talib.ATR(df_medium['High Price'], df_medium['Low Price'], df_medium['Close Price'], timeperiod=21)
df_medium['ATR14'] = talib.ATR(df_medium['High Price'], df_medium['Low Price'], df_medium['Close Price'], timeperiod=14)

In [17]:
df_medium.tail()


,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,ATR21,ATR14
25,1707696000000,48300.00000000,50334.82000000,47710.01000000,49917.27000000,59009.96705000,2024-02-12,1364.699752,1420.146240
26,1707782400000,49917.28000000,50368.61000000,48300.95000000,49699.59000000,55551.56706000,2024-02-13,1398.174050,1466.397223
27,1707868800000,49699.60000000,52043.71000000,49225.01000000,51795.17000000,57046.37401000,2024-02-14,1465.818143,1562.990278
28,1707955200000,51795.17000000,52816.62000000,51314.00000000,51880.00000000,53816.03055000,2024-02-15,1467.570612,1558.678116
29,1708041600000,51880.01000000,52572.08000000,51566.00000000,51951.09000000,33689.40349000,2024-02-16,1445.594869,1519.206822


In [18]:
df_medium['13EMA'] = df_medium['Close Price'].ewm(span=13, adjust=False).mean()

I will choose ATR 21. Now I will create ATR +1, ATR +2, ATR -1 and -2. These are channels. Then we can compare price against these channels to find entry, stop loss and exit.

In [20]:
# Calculate 21-period EMA
df_medium['21EMA'] = df_medium['Close Price'].ewm(span=21, adjust=False).mean()
df_medium.tail()

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,ATR21,ATR14,13EMA,21EMA
25,1707696000000,48300.00000000,50334.82000000,47710.01000000,49917.27000000,59009.96705000,2024-02-12,1364.699752,1420.146240,45572.601815,44469.201929
26,1707782400000,49917.28000000,50368.61000000,48300.95000000,49699.59000000,55551.56706000,2024-02-13,1398.174050,1466.397223,46162.171556,44944.691754
27,1707868800000,49699.60000000,52043.71000000,49225.01000000,51795.17000000,57046.37401000,2024-02-14,1465.818143,1562.990278,46966.885619,45567.462503
28,1707955200000,51795.17000000,52816.62000000,51314.00000000,51880.00000000,53816.03055000,2024-02-15,1467.570612,1558.678116,47668.759102,46141.329548
29,1708041600000,51880.01000000,52572.08000000,51566.00000000,51951.09000000,33689.40349000,2024-02-16,1445.594869,1519.206822,48280.520659,46669.489589


In [22]:
df_medium['ATR+1'] = df_medium['21EMA'] + df_medium['ATR21'] 

In [24]:
df_medium.tail()

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,ATR21,ATR14,13EMA,21EMA,ATR+1
25,1707696000000,48300.00000000,50334.82000000,47710.01000000,49917.27000000,59009.96705000,2024-02-12,1364.699752,1420.146240,45572.601815,44469.201929,45833.901681
26,1707782400000,49917.28000000,50368.61000000,48300.95000000,49699.59000000,55551.56706000,2024-02-13,1398.174050,1466.397223,46162.171556,44944.691754,46342.865803
27,1707868800000,49699.60000000,52043.71000000,49225.01000000,51795.17000000,57046.37401000,2024-02-14,1465.818143,1562.990278,46966.885619,45567.462503,47033.280646
28,1707955200000,51795.17000000,52816.62000000,51314.00000000,51880.00000000,53816.03055000,2024-02-15,1467.570612,1558.678116,47668.759102,46141.329548,47608.900161
29,1708041600000,51880.01000000,52572.08000000,51566.00000000,51951.09000000,33689.40349000,2024-02-16,1445.594869,1519.206822,48280.520659,46669.489589,48115.084458


In [25]:
df_medium['ATR+2'] = df_medium['21EMA'] + 2 * df_medium['ATR21'] 
df_medium['ATR+3'] = df_medium['21EMA'] + 3 * df_medium['ATR21'] 

In [26]:
df_medium.tail()

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,ATR21,ATR14,13EMA,21EMA,ATR+1,ATR+2,ATR+3
25,1707696000000,48300.00000000,50334.82000000,47710.01000000,49917.27000000,59009.96705000,2024-02-12,1364.699752,1420.146240,45572.601815,44469.201929,45833.901681,47198.601434,48563.301186
26,1707782400000,49917.28000000,50368.61000000,48300.95000000,49699.59000000,55551.56706000,2024-02-13,1398.174050,1466.397223,46162.171556,44944.691754,46342.865803,47741.039853,49139.213903
27,1707868800000,49699.60000000,52043.71000000,49225.01000000,51795.17000000,57046.37401000,2024-02-14,1465.818143,1562.990278,46966.885619,45567.462503,47033.280646,48499.098789,49964.916932
28,1707955200000,51795.17000000,52816.62000000,51314.00000000,51880.00000000,53816.03055000,2024-02-15,1467.570612,1558.678116,47668.759102,46141.329548,47608.900161,49076.470773,50544.041385
29,1708041600000,51880.01000000,52572.08000000,51566.00000000,51951.09000000,33689.40349000,2024-02-16,1445.594869,1519.206822,48280.520659,46669.489589,48115.084458,49560.679327,51006.274196


In [27]:
df_medium['ATR-1'] = df_medium['21EMA'] - df_medium['ATR21'] 

In [28]:
df_medium['ATR-2'] = df_medium['21EMA'] - 2 * df_medium['ATR21'] 
df_medium['ATR-3'] = df_medium['21EMA'] - 3 * df_medium['ATR21'] 

In [31]:
df_medium.tail()

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,ATR21,ATR14,13EMA,21EMA,ATR+1,ATR+2,ATR+3,ATR-1,ATR-2,ATR-3
25,1707696000000,48300.00000000,50334.82000000,47710.01000000,49917.27000000,59009.96705000,2024-02-12,1364.699752,1420.146240,45572.601815,44469.201929,45833.901681,47198.601434,48563.301186,43104.502176,41739.802424,40375.102672
26,1707782400000,49917.28000000,50368.61000000,48300.95000000,49699.59000000,55551.56706000,2024-02-13,1398.174050,1466.397223,46162.171556,44944.691754,46342.865803,47741.039853,49139.213903,43546.517704,42148.343654,40750.169604
27,1707868800000,49699.60000000,52043.71000000,49225.01000000,51795.17000000,57046.37401000,2024-02-14,1465.818143,1562.990278,46966.885619,45567.462503,47033.280646,48499.098789,49964.916932,44101.644360,42635.826218,41170.008075
28,1707955200000,51795.17000000,52816.62000000,51314.00000000,51880.00000000,53816.03055000,2024-02-15,1467.570612,1558.678116,47668.759102,46141.329548,47608.900161,49076.470773,50544.041385,44673.758936,43206.188324,41738.617712
29,1708041600000,51880.01000000,52572.08000000,51566.00000000,51951.09000000,33689.40349000,2024-02-16,1445.594869,1519.206822,48280.520659,46669.489589,48115.084458,49560.679327,51006.274196,45223.894721,43778.299852,42332.704983


Ma jätan praegu need kaks tabelit siia, nüüd edasine tegevus
* Määra ära entry, stop ja exit
* Proovi ainult bitcoini peale backtestida
* (Peaks tegema screeneri, mis otsib õigeid stocke, et leiaks selle ostu koha vastavalt Elderi impulsile. )